In [1]:
from database.strategy import Strategy
from database.sec import SEC
from database.market import Market
from strat_tf.premodel_transformer import PreModelTransformer
from strat_tf.model_transformer import ModelTransformer
from strat_tf.modeler import Modeler
from strat_tf.sim_transformer import SimTransformer
from strat_tf.preprocessor import Preprocessor
from utils.date_utils import DateUtils
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from tqdm import tqdm
import tensorflow as tf
import numpy as np
import math

In [2]:
## Loading Constants
start = "2008-01-01"
end = "2020-07-01"
# Loading Databases
strat_db = Strategy("predictive_series_quarterly")
market = Market()
sec = SEC("sec")
market.connect()
sp500 = market.retrieve_data("sp500")
market.close()
reload = False
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
refined = []
refined_quarterly = []
market.connect()
sec.connect()
if reload:
    market.drop_table("dataset_tf")
    for i in tqdm(range(46,47)):
        try:
            ticker = tickers["Symbol"][i]
            cik = int(tickers.iloc[i]["CIK"].item())
            prices = market.retrieve_price_data("prices",ticker)
            filing = sec.retrieve_filing_data(cik)
            prices.reset_index(inplace=True)
            mt = PreModelTransformer(ticker,start,end,90)
            final = mt.fundamental_daily_merge(prices.copy(),filing.copy())
            market.store_data("dataset_tf",final)
        except Exception as e:
            print(str(e),ticker)
market.close()
sec.close()

In [ ]:
prediction_days = 5
batch_size = prediction_days * 3
training_days = batch_size * 5
trading_days = 252
sims = []
sec.connect()
strat_db.connect()
market.connect()
simulation_start = "2011-01-01"
timeline = DateUtils.create_timeline(simulation_start,end)
prices = market.retrieve_data("dataset_regression")
simulation = []
data = market.retrieve_data("dataset_regression")
tickers = [x for x in sp500["Symbol"] if x in data.columns]
training_years = 1
for s in tqdm([1]):
    shift = int(math.ceil(s * 0.69)) 
    for year in range(2015,2021):
        for quarter in range(1,2):
            try:
                month = max(1,(quarter-1)*3)
                prediction_start = datetime(year,quarter,1)
                prediction_end = datetime(year,month+3,1)
                for ticker in ["AAPL"]:
                    try:
                        pt = PreModelTransformer(ticker) 
                        ts = data[(data["date"] <= prediction_start + timedelta(days=s))].iloc[-training_days-shift:-1]
                        training_set = pt.daily_merge(ts,shift)
                        print(shift,training_set["date"].max(),(training_set["date"]-training_set["label_date"]).mean())
                        m = Modeler(ticker,prediction_days)
                        mr = Preprocessor(ticker)
                        refined = mr.preprocess_price_regression(training_set,ticker,batch_size,prediction_days,shift)
                        if len(refined["X"]) > 0:
                            results = m.tf_model(refined)
                            for i in range(10):
                                results_2 = m.tf_model(refined)
                                if (results["results"][1] - results_2["results"][1]) < 0:
                                    break
                                if abs((results["results"][1] - results_2["results"][1]) / results["results"][1]) < 0.01:
                                    break
                                else:
                                    results = results_2
                            model = results["model"]
                            for date in pd.date_range(prediction_start,prediction_end):
                                ps = data[(data["date"] >= date-timedelta(days=batch_size)) & (data["date"] <= date)]
                                prediction_set = pt.daily_prediction_merge(ps)
                                prediction_refined = mr.preprocess_prediction(prediction_set)
                                predictions = model.predict(tf.stack([prediction_refined]))
                                packages = SimTransformer.transform(date,ticker,data,predictions,results,shift)
                                simulation.extend(packages)
#                                 if len(packages) > 0:
#                                     strat_db.store_data("sim_tf",pd.DataFrame(packages))
                    except Exception as e:
                        print(ticker,str(e))
            except Exception as e:
                print(str(e))
sec.close()
market.close()
strat_db.close()

  0%|                                                                                                                                                                                | 0/1 [00:00<?, ?it/s]

1 2014-12-30 00:00:00 -2 days +13:37:17.837837
1/1 [==============================] - 0s 2ms/step - loss: 24.9673 - mean_absolute_percentage_error: 24.9673
1 2015-12-29 00:00:00 -2 days +13:37:17.837837
1/1 [==============================] - 0s 1000us/step - loss: 25.5934 - mean_absolute_percentage_error: 25.5934
1 2016-12-28 00:00:00 -2 days +13:37:17.837837
1/1 [==============================] - 0s 2ms/step - loss: 30.8773 - mean_absolute_percentage_error: 30.8773
1 2017-12-28 00:00:00 -2 days +13:37:17.837837
1/1 [==============================] - 0s 1ms/step - loss: 54.6756 - mean_absolute_percentage_error: 54.6756
1 2018-12-28 00:00:00 -2 days +12:38:55.135135
1/1 [==============================] - 0s 2ms/step - loss: 51.7037 - mean_absolute_percentage_error: 51.7037
1 2019-12-30 00:00:00 -2 days +13:37:17.837837
1/1 [==============================] - 0s 2ms/step - loss: 77.0551 - mean_absolute_percentage_error: 77.0551


In [ ]:
sim = pd.DataFrame(simulation)

In [ ]:
sim

In [ ]:
for shift in sim["shift"].unique():
    print(shift)
    grouped = sim[(sim["sell_date"] <= prediction_end) & (sim["shift"] == shift)].groupby("sell_date").max()
    g = grouped.reset_index().set_index("sell_date")
    plt.plot(g["prediction"])
plt.plot(g["sell_price"])
legend = sim["shift"].unique()
plt.legend(legend)

In [ ]:
predictions

In [ ]:
sim

In [ ]:
grouped